# 1. Imports

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import os
import numpy as np
from torchvision import datasets, models, transforms

from torchvision.models import resnet18, ResNet18_Weights
import torch.optim as optim


import random
import pandas as pd
import pickle

from sklearn.model_selection import train_test_split


from sklearn.model_selection import GroupKFold
from sklearn.model_selection import StratifiedGroupKFold
from imblearn.over_sampling import RandomOverSampler


from codes.utils import stratified_train_test_group_kfold
from codes.utils import model_test_classification
from codes.utils import accuracy_classification
from codes.utils import mlp_reg_cross_val_final_test

from codes.regression_codes import mlp_regression_gridsearch


torch.backends.cudnn.deterministic = True
random.seed(1)
torch.manual_seed(1)
torch.cuda.manual_seed(1)
np.random.seed(1)

use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")
torch.backends.cudnn.benchmark = True

# 2. Data

In [2]:
with open("Data/MoS2_Analysis_Data_trained2", "rb") as fp:   # Unpickling
    MoS2_Proj1_Class_Data = pickle.load(fp)

#features = MoS2_ImageNet_100_data[0]
df = pd.DataFrame(MoS2_Proj1_Class_Data)
print(df.keys())
T_target = np.array(list(df['T']))
features = np.array(list(df['ImageNet']))
sampleId = np.array(list(df['sampleId']))


X = features
Y = np.array(T_target)

groups = np.array(sampleId)


train_val_groups, train_val_X, train_val_Y, test_X, test_Y = stratified_train_test_group_kfold(X, Y, groups, n_splits=10, test_fold=0)


#train_val_Y = train_val_Y.flatten()
#val_Y = val_Y.flatten()
#test_Y = test_Y.flatten()


print(train_val_X.shape)
print(train_val_Y.shape)
print(test_X.shape)
print(test_Y.shape)

oversample = RandomOverSampler(sampling_strategy='not majority')




Index(['sampleId', 'sampleLabel', 'image', 'T', 'ImageNet', 'MicroNet'], dtype='object')
<class 'generator'>
(235, 100)
(235,)
(27, 100)
(27,)


# 3. Runing Cross-Validation

In [3]:
!mkdir -p 'Models/regression/ImageNet/aug3/MLP'

In [4]:
# the paramters below were optimized
#lr: 0.01, batch size: 16, dropout: 0.35, epochs: 32, l1: 600, l2: 50
                        
                        
Learning_rate = [0.005]
Drop_out = [0.00]#[0.0, 0.2, 0.35, 0.5]
Batch_size = [32]#[16, 32, 100]
L1 = [400]#[600, 400, 300, 200]
L2 = [100]# [150, 100, 50]

def cross_10_folds_mlp(train_val_X, train_val_Y):
    #train_val_Y.reshape(len(train_val_Y),1)

    best_train = []
    best_val = []
    best_variables = []
    best_performance_record = []
    for fold in range(10):
        
        model_path = f'regression/ImageNet/aug3/MLP/{fold}_model.pth'
        group, train_X, train_Y, val_X, val_Y = stratified_train_test_group_kfold(train_val_X, train_val_Y, train_val_groups, n_splits=10, test_fold=fold)
        #train_X, val_X, train_Y, val_Y = train_test_split(train_val_X, train_val_Y, test_size=0.1,stratify=train_val_Y,random_state=fold+30)#42
        train_X, train_Y = oversample.fit_resample(train_X, train_Y)
    
        best_train_acc, best_val_acc, best_record, hyper=mlp_regression_gridsearch(train_X, train_Y, val_X, val_Y,Learning_rate, L1, L2, Drop_out,Batch_size, fold,model_path)
        #best_train_acc, best_val_acc, trainf1, valf1, best_perf_record, hyper=cnn_class_train_fn(train_X, train_Y, val_X, val_Y, Learning_rate, Batch_size, fold)

        best_train.append(best_train_acc)
        best_val.append(best_val_acc)
        best_variables.append(hyper)
        best_performance_record.append(best_record)

        print(f'fold: {fold} done!')
    return best_train, best_val, best_variables, best_performance_record


best_train, best_val, best_variables, best_performance_record = cross_10_folds_mlp(train_val_X, train_val_Y)



<class 'generator'>
Epoch1: loss: 846912.8750 val_loss: 675364.5000
Epoch2: loss: 331703.5553 val_loss: 26980.2324
Epoch3: loss: 94075.4768 val_loss: 10814.2441
Epoch4: loss: 40751.1094 val_loss: 25436.9980
Epoch5: loss: 20810.0592 val_loss: 23927.9180
Epoch6: loss: 16438.7070 val_loss: 11764.0527
Epoch7: loss: 13140.1489 val_loss: 11098.8203
Epoch8: loss: 11089.7662 val_loss: 9432.6035
Epoch9: loss: 10264.8479 val_loss: 9677.7695
Epoch10: loss: 9268.7883 val_loss: 8931.3770
Epoch11: loss: 8572.7666 val_loss: 9113.9512
Epoch12: loss: 7978.0914 val_loss: 8790.5850
Epoch13: loss: 7646.9170 val_loss: 8644.7949
Epoch14: loss: 7203.4300 val_loss: 7915.0332
Epoch15: loss: 6829.4598 val_loss: 8515.2607
Epoch16: loss: 6604.4916 val_loss: 7455.2812
Epoch17: loss: 6163.1127 val_loss: 7950.0894
Epoch18: loss: 6027.4669 val_loss: 6883.5381
Epoch19: loss: 5724.0151 val_loss: 7010.8594
Epoch20: loss: 5474.7882 val_loss: 6407.1250
Epoch21: loss: 5523.0283 val_loss: 6891.2554
Epoch22: loss: 5185.6632 

# 4. Test Models

In [5]:
class MLP(nn.Module):
    def __init__(self, l1=120, l2=84, p = 0.2):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(100, l1)
        self.fc2 = nn.Linear(l1, l2)
        self.fc3 = nn.Linear(l2, 1)
        #self.activ = torch.nn.Sigmoid()
        self.dropout = nn.Dropout(p)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)
        return x

l1, l2, p = 400, 100, 0.0
trained_model = model = MLP(l1, l2, p).to(device)

In [7]:

#trained_model = pretrained_model()
root_path = f'regression/ImageNet/aug3/MLP'

best_test, confusion_matrix_test, rmse_test_folds = mlp_reg_cross_val_final_test(trained_model, test_X, test_Y, 'test', root_path)


print(best_test)
print(f'test_acc_mean: {np.mean(best_test) :.3f}, std: {np.std(best_test) :.3f}')
print("......")

def cm_to_plot(best_test):

    max_test = np.max(best_test)
    idx_cm = []
    for index, item in enumerate(best_test):
        if item == max_test:
            idx_cm.append(index)

    print('idx_cm: ', idx_cm)
    print('max_test_acc: ', max_test)

    print(confusion_matrix_test[idx_cm[0]])
cm_to_plot(best_test)
print('5th confusion matrix: ', [list(item) for item in confusion_matrix_test[4]])


cm_list = []
for cm in confusion_matrix_test:
    cm_list.append([list(item) for item in cm])
#print(cm_list)

print(len(cm_list))
print('avg cm: ', [list(item) for item in np.mean(cm_list, axis=0)])


Accuracy of the network on the 27 test images: 37.0 %
Accuracy of the network on the 27 test images: 40.7 %
Accuracy of the network on the 27 test images: 37.0 %
Accuracy of the network on the 27 test images: 48.1 %
Accuracy of the network on the 27 test images: 44.4 %
Accuracy of the network on the 27 test images: 29.6 %
Accuracy of the network on the 27 test images: 44.4 %
Accuracy of the network on the 27 test images: 40.7 %
Accuracy of the network on the 27 test images: 48.1 %
Accuracy of the network on the 27 test images: 48.1 %
[37.03703703703704, 40.74074074074074, 37.03703703703704, 48.148148148148145, 44.44444444444444, 29.62962962962963, 44.44444444444444, 40.74074074074074, 48.148148148148145, 48.148148148148145]
test_acc_mean: 41.852, std: 5.750
......
idx_cm:  [3, 8, 9]
max_test_acc:  48.148148148148145
[[1 1 1]
 [2 7 4]
 [3 3 5]]
5th confusion matrix:  [[1, 1, 1], [2, 6, 5], [3, 3, 5]]
10
avg cm:  [[1.1, 0.9, 1.0], [3.2, 5.3, 4.5], [3.9, 2.2, 4.9]]
